In [6]:
import pandas as pd
import numpy as np
import os
import glob
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.model_selection import train_test_split,GridSearchCV
import pickle
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.metrics import accuracy_score, roc_curve, precision_score, recall_score, confusion_matrix, roc_auc_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import json
# from sklearn.ensemble import CatBoostClassifier
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.font_manager
from pyod.models.knn import KNN 
from pyod.utils.data import generate_data, get_outliers_inliers

In [229]:
client = bigquery.Client()

print("Client creating using default project: {}".format(client.project))
query = """
 select * from `jm-ebg.EDA_MODEL.lable_data_with_extra_features`
 """
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    # location="US",
)  # API request - starts the query
df = query_job.to_dataframe()
df.shape
# df.to_csv('train_data.csv')

Client creating using default project: jm-ebg


(102571, 28)

In [230]:
df.head()

,ProspectId,user_pseudo_id,visitStartTime,event_date,timeOnSite_past,day_of_week_past,device_category_past,mobile_brand_name_past,operating_system_past,os_version_past,...,ProspectId_conv,visitStartTime_conv,date_conv,day_diff,label,ProspectID_LeadCreation,mx_Last_Activity_Notes,ProspectActivityName_Max,mx_Last_Activity_done_by_Agent,mx_Sub_Dispostion
0,00d2aaa0-69b1-4b8f-89b8-e5f9b53a8b6a,711925274.1675325695,1675325695,20230202,0.682301,5,desktop,Opera Software,Windows,Windows 7,...,None,<NA>,None,<NA>,0,00d2aaa0-69b1-4b8f-89b8-e5f9b53a8b6a,busy,Not Contactable,Not Contactable,Others
1,01304386-f4b0-4601-ba81-097b966095fe,2061620715.1671277346,1675582360,20230205,35.725837,1,mobile,OPPO,Android,Android 12,...,None,<NA>,None,<NA>,0,01304386-f4b0-4601-ba81-097b966095fe,call disconnected by client,Outbound Phone Call Activity,Not Contactable,Call Disconnected By Client - No Communication
2,019e7323-c132-4cf3-b97e-048b84ed435a,222679061.1675254861,1675254861,20230201,0.462754,4,mobile,Realme,Android,Android 11,...,None,<NA>,None,<NA>,0,019e7323-c132-4cf3-b97e-048b84ed435a,busy,Outbound Phone Call Activity,Not Contactable,Others
3,01fbcabe-8ce6-4234-8ee6-e616fdf22136,1529146629.1673193933,1677211898,20230224,24.002012,6,mobile,Vivo,Android,Android 11,...,None,<NA>,None,<NA>,0,01fbcabe-8ce6-4234-8ee6-e616fdf22136,intrested,Outbound Phone Call Activity,Call Back,intrested
4,0294d226-f9e7-4ebe-a176-2f7cb5eae5ed,1396493678.1675044228,1678425248,20230310,4.844952,6,mobile,Vivo,Android,Android 12.0.0,...,None,<NA>,None,<NA>,0,0294d226-f9e7-4ebe-a176-2f7cb5eae5ed,call drop,Not Contactable,Not Contactable,Call Disconnected By Client - No Communication


In [231]:
df['ft_campaign_past'].value_counts()

(direct)                                                  56386
link                                                      11015
Leads_Performance_Demat_Tier_2                             5062
Leads_Performance_Demat_Tier_1                             4675
Leads_Performance_Max_Advisory_Tier_1                      3260
                                                          ...  
PT_Conversion_Tier1_above_24_Interest_1_above24 – Copy        1
Leads_Performance_Demat_Tier1                                 1
pennystocks                                                   1
24041                                                         1
investingmistakes                                             1
Name: ft_campaign_past, Length: 142, dtype: int64

In [232]:
int_cols = list(df.select_dtypes(include=[int]).columns)
int_cols.remove('label')

In [233]:
cat_cols = list(df.select_dtypes(include=['O']).columns)
float_cols = list(df.select_dtypes(include=[float]).columns)

In [234]:
data={}
unwanted = {'(none)','(direct)','(not set)','(Other)'}  

In [235]:
def analysis_count(column_name,number):
    return df.groupby([column_name])[column_name].size().reset_index(name='count') \
                     .sort_values(['count'], ascending=False) \
                     .head(number).reset_index(drop=True)
def analysis_column(column_name,number):
    return df.groupby([column_name])[column_name].size().reset_index(name='count') \
                     .sort_values(['count'], ascending=False) \
                     .head(number)[column_name].tolist()

In [236]:
##data -> dictionary 
# data['placement'] =  analysis_column("placement",150)
# data['placement'] = [i for i in data['placement']  if i not in unwanted]
data['device_category_past'] =  analysis_column('device_category_past',4)
data['device_category_past'] = [i for i in data['device_category_past']  if i not in unwanted] 
data['mobile_brand_name_past'] =  analysis_column('mobile_brand_name_past',10)
data['mobile_brand_name_past'] = [i for i in data['mobile_brand_name_past']  if i not in unwanted] 
data['operating_system_past'] =  analysis_column('operating_system_past',5)
data['operating_system_past'] = [i for i in data['operating_system_past']  if i not in unwanted] 
data['os_version_past'] =  analysis_column('os_version_past',10)
data['os_version_past'] = [i for i in data['os_version_past']  if i not in unwanted] 
data['browser_past'] =  analysis_column('browser_past',8)
data['browser_past'] = [i for i in data['browser_past']  if i not in unwanted]
data['city_past'] =  analysis_column('city_past',160)
data['city_past'] = [i for i in data['city_past']  if i not in unwanted]
data['region_past'] =  analysis_column('region_past',50)
data['region_past'] = [i for i in data['region_past']  if i not in unwanted] 
data['ft_campaign_past'] =  analysis_column('ft_campaign_past',70)
data['ft_campaign_past'] = [i for i in data['ft_campaign_past']  if i not in unwanted]
data['ft_medium_past'] =  analysis_column('ft_medium_past',5)
data['ft_medium_past'] = [i for i in data['ft_medium_past']  if i not in unwanted] 
data['ft_source_past'] =  analysis_column('ft_source_past',10)
data['ft_source_past'] = [i for i in data['ft_source_past']  if i not in unwanted] 
data['utm_term_placement_past'] =  analysis_column('utm_term_placement_past',200)
data['utm_term_placement_past'] = [i for i in data['utm_term_placement_past']  if i not in unwanted]
# data['placement_past'] =  analysis_column('placement_past',150)
# data['placement_past'] = [i for i in data['placement_past']  if i not in unwanted]

In [237]:
# df['placement'].loc[df['placement'].apply(lambda x: (x not in data['placement']) )] = '(Others)'
df['device_category_past'].loc[df['device_category_past'].apply(lambda x: (x not in data['device_category_past']) )] = '(Others)'
df['mobile_brand_name_past'].loc[df['mobile_brand_name_past'].apply(lambda x: (x not in data['mobile_brand_name_past']) )] = '(Others)'
df['operating_system_past'].loc[df['operating_system_past'].apply(lambda x: (x not in data['operating_system_past']) )] = '(Others)'
df['os_version_past'].loc[df['os_version_past'].apply(lambda x: (x not in data['os_version_past']) )] = '(Others)'
df['browser_past'].loc[df['browser_past'].apply(lambda x: (x not in data['browser_past']) )] = '(Others)'
df['city_past'].loc[df['city_past'].apply(lambda x: (x not in data['city_past']) )] = '(Others)'
df['region_past'].loc[df['region_past'].apply(lambda x: (x not in data['region_past']) )] = '(Others)'
df['ft_campaign_past'].loc[df['ft_campaign_past'].apply(lambda x: (x not in data['ft_campaign_past']) )] = '(Others)'
df['ft_medium_past'].loc[df['ft_medium_past'].apply(lambda x: (x not in data['ft_medium_past']) )] = '(Others)'
df['ft_source_past'].loc[df['ft_source_past'].apply(lambda x: (x not in data['ft_source_past']) )] = '(Others)'
df['utm_term_placement_past'].loc[df['utm_term_placement_past'].apply(lambda x: (x not in data['utm_term_placement_past']) )] = '(Others)'
# df['placement_past'].loc[df['placement_past'].apply(lambda x: (x not in data['placement_past']) )] = '(Others)'

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [238]:
df.head()

,ProspectId,user_pseudo_id,visitStartTime,event_date,timeOnSite_past,day_of_week_past,device_category_past,mobile_brand_name_past,operating_system_past,os_version_past,...,ProspectId_conv,visitStartTime_conv,date_conv,day_diff,label,ProspectID_LeadCreation,mx_Last_Activity_Notes,ProspectActivityName_Max,mx_Last_Activity_done_by_Agent,mx_Sub_Dispostion
0,00d2aaa0-69b1-4b8f-89b8-e5f9b53a8b6a,711925274.1675325695,1675325695,20230202,0.682301,5,desktop,(Others),Windows,(Others),...,None,<NA>,None,<NA>,0,00d2aaa0-69b1-4b8f-89b8-e5f9b53a8b6a,busy,Not Contactable,Not Contactable,Others
1,01304386-f4b0-4601-ba81-097b966095fe,2061620715.1671277346,1675582360,20230205,35.725837,1,mobile,OPPO,Android,Android 12,...,None,<NA>,None,<NA>,0,01304386-f4b0-4601-ba81-097b966095fe,call disconnected by client,Outbound Phone Call Activity,Not Contactable,Call Disconnected By Client - No Communication
2,019e7323-c132-4cf3-b97e-048b84ed435a,222679061.1675254861,1675254861,20230201,0.462754,4,mobile,Realme,Android,Android 11,...,None,<NA>,None,<NA>,0,019e7323-c132-4cf3-b97e-048b84ed435a,busy,Outbound Phone Call Activity,Not Contactable,Others
3,01fbcabe-8ce6-4234-8ee6-e616fdf22136,1529146629.1673193933,1677211898,20230224,24.002012,6,mobile,Vivo,Android,Android 11,...,None,<NA>,None,<NA>,0,01fbcabe-8ce6-4234-8ee6-e616fdf22136,intrested,Outbound Phone Call Activity,Call Back,intrested
4,0294d226-f9e7-4ebe-a176-2f7cb5eae5ed,1396493678.1675044228,1678425248,20230310,4.844952,6,mobile,Vivo,Android,Android 12.0.0,...,None,<NA>,None,<NA>,0,0294d226-f9e7-4ebe-a176-2f7cb5eae5ed,call drop,Not Contactable,Not Contactable,Call Disconnected By Client - No Communication


In [239]:
### replace the string int and float value
for i in cat_cols:
    df[i].fillna("Value_Not_Found", inplace = True)
for i in int_cols:
    df[i].fillna(-99999, inplace = True) 
for i in float_cols:
    df[i].fillna(-99999999.000, inplace = True) 

In [240]:
drop_cols = ['user_pseudo_id', 'visitStartTime', 'event_date','user_psudo_id_conv', 'ProspectId_conv', 'visitStartTime_conv',
       'date_conv', 'day_diff','device_category_past','ProspectID_LeadCreation','mx_Last_Activity_Notes','ProspectId']
df = df.drop(drop_cols, axis=1)

In [241]:
df['label'].value_counts()

0    91004
1    11567
Name: label, dtype: Int64

In [242]:
df.columns

Index(['timeOnSite_past', 'day_of_week_past', 'mobile_brand_name_past',
       'operating_system_past', 'os_version_past', 'browser_past',
       'region_past', 'city_past', 'ft_campaign_past', 'ft_medium_past',
       'ft_source_past', 'utm_term_placement_past', 'label',
       'ProspectActivityName_Max', 'mx_Last_Activity_done_by_Agent',
       'mx_Sub_Dispostion'],
      dtype='object')

In [243]:
cat_cols = list(df.select_dtypes(include=['O']).columns)
cat_cols

['mobile_brand_name_past',
 'operating_system_past',
 'os_version_past',
 'browser_past',
 'region_past',
 'city_past',
 'ft_campaign_past',
 'ft_medium_past',
 'ft_source_past',
 'utm_term_placement_past',
 'ProspectActivityName_Max',
 'mx_Last_Activity_done_by_Agent',
 'mx_Sub_Dispostion']

In [244]:
#categorical value handling
le = LabelEncoder()
for  i in cat_cols:
    df[i] = le.fit_transform(df[i])
    pickle.dump(le, open("label_encode/{}-esd-RF.pkl".format(i), 'wb'))

In [313]:
X1.columns

Index(['timeOnSite_past', 'day_of_week_past', 'mobile_brand_name_past',
       'operating_system_past', 'os_version_past', 'browser_past',
       'region_past', 'city_past', 'ft_campaign_past', 'ft_medium_past',
       'ft_source_past', 'utm_term_placement_past', 'ProspectActivityName_Max',
       'mx_Last_Activity_done_by_Agent', 'mx_Sub_Dispostion'],
      dtype='object')

In [245]:
X1= df.drop(['label'], axis=1)
y1 = df.label

In [246]:
y1=np.floor(pd.to_numeric(y1, errors='coerce')).astype('Int64')
import numpy as np
y1 = y1.fillna(0).astype(np.int64, errors='ignore')
X1 = X1.fillna(0).astype(np.int64, errors='ignore')

In [247]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SVMSMOTE
oversample = SMOTE()
X1, y1 = oversample.fit_resample(X1, y1)

In [248]:
X_train, X_test, y_train, y_test=train_test_split(X1, y1, test_size=.25, stratify=y1)

In [249]:
from sklearn.preprocessing import StandardScaler

ss_train = MinMaxScaler()
X_train = ss_train.fit_transform(X_train)

ss_test = MinMaxScaler()
X_test = ss_test.fit_transform(X_test)

In [251]:
#Fitting Logistic Regression to the training set  
from sklearn.linear_model import LogisticRegression  
classifier= LogisticRegression(random_state=0,max_iter=1000000)  
classifier.fit(X_train, y_train) 

LogisticRegression(max_iter=1000000, random_state=0)

In [310]:
filename = 'model/Dhawal_v1.pkl'
pickle.dump(classifier, open(filename, 'wb'))

In [312]:
classifier.feature_names_in_

AttributeError: 'LogisticRegression' object has no attribute 'feature_names_in_'

In [252]:
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
from sklearn.calibration import CalibratedClassifierCV
logreg = LinearSVC(max_iter=1000000)
model = CalibratedClassifierCV(logreg) 
model.fit(X_train, y_train) 

CalibratedClassifierCV(base_estimator=LinearSVC(max_iter=1000000))

In [254]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score, f1_score
y_pred=classifier.predict(X_test)
print("accuracy : ", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print("roc auc : ", roc_auc_score(y_test, y_pred))
print("recall : ", recall_score(y_test, y_pred))
print("precision : ", precision_score(y_test, y_pred))
print("f1 score : ",f1_score(y_test,y_pred) )

print ("----------------------")
y_pred=model.predict(X_test)
print("accuracy : ", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print("roc auc : ", roc_auc_score(y_test, y_pred))
print("recall : ", recall_score(y_test, y_pred))
print("precision : ", precision_score(y_test, y_pred))
print("f1 score : ",f1_score(y_test,y_pred) )


accuracy :  0.8517647575930728
[[19401  3350]
 [ 3395 19356]]
roc auc :  0.8517647575930729
recall :  0.8507757900751616
precision :  0.8524619043424645
f1 score :  0.8516180126273181
----------------------
accuracy :  0.8522922069359589
[[19647  3104]
 [ 3617 19134]]
roc auc :  0.8522922069359588
recall :  0.8410179772317701
precision :  0.8604191024372695
f1 score :  0.8506079263820044


In [ ]:
clf1


In [56]:
filename = 'model/RF_Dhawal.pkl'

pickle.dump(clf1, open(filename, 'wb'))

In [41]:
#Predicting the test set result  
y_pred= model.predict(X) 
y_pred1= model.predict_proba(X) 

In [42]:
y_pred=y

In [43]:
y_pred1=y_pred1[:,1]

In [44]:
y_pred=list(y_pred)

In [45]:
y_pred1
for i in range(y_pred1.size):
    if y_pred1[i]>0.7250:
        y_pred1[i]=1
    elif y_pred1[i]<0.2550:
        y_pred1[i]=0
    else:
        y_pred1[i]=-1

In [46]:
y_pred1

array([ 1.,  0.,  1., ...,  0.,  1., -1.])

In [47]:
a=[]
b=[]
for i in range(y_pred1.size):
    if y_pred1[i]==1:
        a.append(y_pred1[i])
        b.append(y_pred[i])
    elif y_pred1[i]==0:
        a.append(y_pred1[i])
        b.append(y_pred[i])
        

In [48]:
len(a)

10723

In [49]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(a, b)

TN, FP, FN, TP = confusion_matrix(a, b).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)

True Positive(TP)  =  232
False Positive(FP) =  123
True Negative(TN)  =  6049
False Negative(FN) =  4319


In [50]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score, f1_score



print ("----------------------")


print("accuracy : ", accuracy_score(a, b))
print(confusion_matrix(a, b))
print("roc auc : ", roc_auc_score(a, b))
print("recall : ", recall_score(a, b))
print("precision : ", precision_score(a, b))


----------------------
accuracy :  0.5857502564580808
[[6049  123]
 [4319  232]]
roc auc :  0.5155245483853833
recall :  0.05097780707536805
precision :  0.6535211267605634


In [70]:
f1_score(a,b)

0.09457806767223807

# Production 

In [9]:
client = bigquery.Client()

print("Client creating using default project: {}".format(client.project))
query = """
 SELECT * FROM `jm-ebg.Try_Test_Data.may_label_1_10_final`
 """
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    # location="US",
)  # API request - starts the query
df3 = query_job.to_dataframe()
df3.shape
# df.to_csv('train_data.csv')

Client creating using default project: jm-ebg


(19268, 30)

In [10]:
int_cols = list(df3.select_dtypes(include=[int]).columns)
int_cols.remove('label')

In [11]:
cat_cols = list(df3.select_dtypes(include=['O']).columns)
float_cols = list(df3.select_dtypes(include=[float]).columns)

In [12]:
data={}
unwanted = {'(none)','(direct)','(not set)','(Other)'}  

In [13]:
def analysis_count(column_name,number):
    return df3.groupby([column_name])[column_name].size().reset_index(name='count') \
                     .sort_values(['count'], ascending=False) \
                     .head(number).reset_index(drop=True)
def analysis_column(column_name,number):
    return df3.groupby([column_name])[column_name].size().reset_index(name='count') \
                     .sort_values(['count'], ascending=False) \
                     .head(number)[column_name].tolist()

In [14]:
##data -> dictionary 
# data['placement'] =  analysis_column("placement",150)
# data['placement'] = [i for i in data['placement']  if i not in unwanted]
data['device_category_past'] =  analysis_column('device_category_past',4)
data['device_category_past'] = [i for i in data['device_category_past']  if i not in unwanted] 
data['mobile_brand_name_past'] =  analysis_column('mobile_brand_name_past',10)
data['mobile_brand_name_past'] = [i for i in data['mobile_brand_name_past']  if i not in unwanted] 
data['operating_system_past'] =  analysis_column('operating_system_past',5)
data['operating_system_past'] = [i for i in data['operating_system_past']  if i not in unwanted] 
data['os_version_past'] =  analysis_column('os_version_past',10)
data['os_version_past'] = [i for i in data['os_version_past']  if i not in unwanted] 
data['browser_past'] =  analysis_column('browser_past',8)
data['browser_past'] = [i for i in data['browser_past']  if i not in unwanted]
data['city_past'] =  analysis_column('city_past',160)
data['city_past'] = [i for i in data['city_past']  if i not in unwanted]
data['region_past'] =  analysis_column('region_past',50)
data['region_past'] = [i for i in data['region_past']  if i not in unwanted] 
data['ft_campaign_past'] =  analysis_column('ft_campaign_past',70)
data['ft_campaign_past'] = [i for i in data['ft_campaign_past']  if i not in unwanted]
data['ft_medium_past'] =  analysis_column('ft_medium_past',5)
data['ft_medium_past'] = [i for i in data['ft_medium_past']  if i not in unwanted] 
data['ft_source_past'] =  analysis_column('ft_source_past',10)
data['ft_source_past'] = [i for i in data['ft_source_past']  if i not in unwanted] 
data['utm_term_placement_past'] =  analysis_column('utm_term_placement_past',200)
data['utm_term_placement_past'] = [i for i in data['utm_term_placement_past']  if i not in unwanted]
# data['placement_past'] =  analysis_column('placement_past',150)
# data['placement_past'] = [i for i in data['placement_past']  if i not in unwanted]

In [15]:
# df['placement'].loc[df['placement'].apply(lambda x: (x not in data['placement']) )] = '(Others)'
df3['device_category_past'].loc[df3['device_category_past'].apply(lambda x: (x not in data['device_category_past']) )] = '(Others)'
df3['mobile_brand_name_past'].loc[df3['mobile_brand_name_past'].apply(lambda x: (x not in data['mobile_brand_name_past']) )] = '(Others)'
df3['operating_system_past'].loc[df3['operating_system_past'].apply(lambda x: (x not in data['operating_system_past']) )] = '(Others)'
df3['os_version_past'].loc[df3['os_version_past'].apply(lambda x: (x not in data['os_version_past']) )] = '(Others)'
df3['browser_past'].loc[df3['browser_past'].apply(lambda x: (x not in data['browser_past']) )] = '(Others)'
df3['city_past'].loc[df3['city_past'].apply(lambda x: (x not in data['city_past']) )] = '(Others)'
df3['region_past'].loc[df3['region_past'].apply(lambda x: (x not in data['region_past']) )] = '(Others)'
df3['ft_campaign_past'].loc[df3['ft_campaign_past'].apply(lambda x: (x not in data['ft_campaign_past']) )] = '(Others)'
df3['ft_medium_past'].loc[df3['ft_medium_past'].apply(lambda x: (x not in data['ft_medium_past']) )] = '(Others)'
df3['ft_source_past'].loc[df3['ft_source_past'].apply(lambda x: (x not in data['ft_source_past']) )] = '(Others)'
df3['utm_term_placement_past'].loc[df3['utm_term_placement_past'].apply(lambda x: (x not in data['utm_term_placement_past']) )] = '(Others)'
# df['placement_past'].loc[df['placement_past'].apply(lambda x: (x not in data['placement_past']) )] = '(Others)'

In [16]:
### replace the string int and float value
for i in cat_cols:
    df3[i].fillna("Value_Not_Found", inplace = True)
for i in int_cols:
    df3[i].fillna(-99999, inplace = True) 
for i in float_cols:
    df3[i].fillna(-99999999.000, inplace = True) 

In [17]:
#categorical value handling
le = LabelEncoder()
for  i in cat_cols:
    df3[i] = le.fit_transform(df3[i])
    pickle.dump(le, open("label_encode/{}-esd-RF.pkl".format(i), 'wb'))

In [20]:
df3.columns

Index(['ProspectID_Past', 'user_pseudo_id_past', 'visitStartTime_past',
       'date_past', 'timeOnSite_past', 'day_of_week_past',
       'device_category_past', 'mobile_brand_name_past',
       'operating_system_past', 'os_version_past', 'browser_past',
       'region_past', 'city_past', 'ft_campaign_past', 'ft_medium_past',
       'ft_source_past', 'ad_network_past', 'ad_group_past',
       'utm_term_placement_past', 'user_psudo_id_conv', 'ProspectId_conv',
       'visitStartTime_conv', 'date_conv', 'day_diff', 'label',
       'ProspectID_LeadCreation', 'mx_Last_Activity_Notes',
       'ProspectActivityName_Max', 'mx_Last_Activity_done_by_Agent',
       'mx_Sub_Dispostion'],
      dtype='object')

In [22]:
drop_cols = ['date_past','ProspectID_Past','device_category_past',
             'mx_Last_Activity_Notes','user_pseudo_id_past','visitStartTime_past',
            'ad_network_past', 'ad_group_past','user_psudo_id_conv', 'ProspectId_conv',
       'visitStartTime_conv', 'date_conv', 'day_diff','mx_Last_Activity_Notes']
df3 = df3.drop(drop_cols, axis=1)

In [23]:
df3['label'].value_counts()

0    18811
1      457
Name: label, dtype: Int64

In [51]:
X = df3.drop(['label'], axis=1)
y = df3.label

In [52]:
X=X.loc[:,['timeOnSite_past', 'day_of_week_past', 'mobile_brand_name_past',
       'operating_system_past', 'os_version_past', 'browser_past',
       'region_past', 'city_past', 'ft_campaign_past', 'ft_medium_past',
       'ft_source_past', 'utm_term_placement_past',
        'ProspectActivityName_Max',
       'mx_Last_Activity_done_by_Agent', 'mx_Sub_Dispostion']]

In [53]:
X.columns

Index(['timeOnSite_past', 'day_of_week_past', 'mobile_brand_name_past',
       'operating_system_past', 'os_version_past', 'browser_past',
       'region_past', 'city_past', 'ft_campaign_past', 'ft_medium_past',
       'ft_source_past', 'utm_term_placement_past', 'ProspectActivityName_Max',
       'mx_Last_Activity_done_by_Agent', 'mx_Sub_Dispostion'],
      dtype='object')

In [54]:
df.columns

Index(['ProspectID_Past', 'user_pseudo_id_past', 'visitStartTime_past',
       'date_past', 'timeOnSite_past', 'day_of_week_past',
       'device_category_past', 'mobile_brand_name_past',
       'operating_system_past', 'os_version_past', 'browser_past',
       'region_past', 'city_past', 'ft_campaign_past', 'ft_medium_past',
       'ft_source_past', 'ad_network_past', 'ad_group_past',
       'utm_term_placement_past', 'user_psudo_id_conv', 'ProspectId_conv',
       'visitStartTime_conv', 'date_conv', 'day_diff', 'label',
       'ProspectID_LeadCreation', 'mx_Last_Activity_Notes',
       'ProspectActivityName_Max', 'mx_Last_Activity_done_by_Agent',
       'mx_Sub_Dispostion'],
      dtype='object')

In [55]:
y=np.floor(pd.to_numeric(y, errors='coerce')).astype('Int64')
import numpy as np
y = y.fillna(0).astype(np.int64, errors='ignore')
X = X.fillna(0).astype(np.int64, errors='ignore')

In [56]:
ss_train = MinMaxScaler()
X= ss_train.fit_transform(X)

In [57]:
filename = 'model/Dhawal_v1.pkl'
model = pickle.load(open(filename, 'rb'))

In [69]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score, f1_score

y_pred = model.predict(X)
#y_pred = [1 if val >= 0.50 else 0 for val in y_pred_prob[:,1]]
f1 = f1_score(y, y_pred)
print("accuracy : ", accuracy_score(y, y_pred))
print(confusion_matrix(y, y_pred))
print("roc auc : ", roc_auc_score(y, y_pred))
print("recall : ", recall_score(y, y_pred))
print("precision : ", precision_score(y, y_pred))
print("f1 score : ",f1 )


accuracy :  0.5594768528129541
[[10493  8318]
 [  170   287]]
roc auc :  0.59291033564676
recall :  0.6280087527352297
precision :  0.03335270191748983
f1 score :  0.06334142573383358


In [60]:
y=y.astype('int64')

In [61]:
y_pred= model.predict(X) 
y_pred1= model.predict_proba(X) 

In [62]:
y_pred=y

In [63]:
y_pred1=y_pred1[:,1]

In [64]:
y_pred=list(y_pred)

In [65]:
y_pred1
for i in range(y_pred1.size):
    if y_pred1[i]>0.7250:
        y_pred1[i]=1
    elif y_pred1[i]<0.2550:
        y_pred1[i]=0
    else:
        y_pred1[i]=-1

In [66]:
a=[]
b=[]
for i in range(y_pred1.size):
    if y_pred1[i]==1:
        a.append(y_pred1[i])
        b.append(y_pred[i])
    elif y_pred1[i]==0:
        a.append(y_pred1[i])
        b.append(y_pred[i])

In [67]:
len(a)

10723

In [68]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score, f1_score



print ("----------------------")


print("accuracy : ", accuracy_score(a, b))
print(confusion_matrix(a, b))
print("roc auc : ", roc_auc_score(a, b))
print("recall : ", recall_score(a, b))
print("precision : ", precision_score(a, b))

----------------------
accuracy :  0.5857502564580808
[[6049  123]
 [4319  232]]
roc auc :  0.5155245483853833
recall :  0.05097780707536805
precision :  0.6535211267605634


In [74]:
df3.head()

,day_of_week_past,timeOnSite_past,mobile_brand_name_past,operating_system_past,os_version_past,browser_past,region_past,city_past,ft_campaign_past,ft_source_past,ft_medium_past,utm_term_placement_past,ProspectActivityName_Max,mx_Last_Activity_done_by_Agent,mx_Sub_Dispostion,label
0,3,0.000000,3,0,3,2,4,68,0,0,0,0,9,3,46,0
1,2,1.474086,0,0,0,2,30,47,15,6,1,0,10,5,40,0
2,2,0.188153,0,0,0,2,30,47,15,6,1,0,10,5,40,0
3,2,1.897437,0,0,0,2,30,47,15,6,1,0,10,5,40,0
4,2,21.324201,0,0,0,2,30,47,15,6,1,0,10,5,40,0


In [75]:
df.head()

,timeOnSite_past,day_of_week_past,mobile_brand_name_past,operating_system_past,os_version_past,browser_past,region_past,city_past,ft_campaign_past,ft_medium_past,ft_source_past,utm_term_placement_past,label,ProspectActivityName_Max,mx_Last_Activity_done_by_Agent,mx_Sub_Dispostion
0,0.682301,5,0,4,0,5,46,3,48,1,6,71,0,12,8,62
1,35.725837,1,4,1,5,2,26,102,0,0,0,0,0,15,8,20
2,0.462754,4,6,1,3,2,42,32,39,1,6,142,0,15,8,62
3,24.002012,6,9,1,3,2,43,61,65,1,6,0,0,15,2,113
4,4.844952,6,9,1,6,2,26,102,0,0,0,0,0,12,8,20


In [79]:
X.head()

,timeOnSite_past,day_of_week_past,mobile_brand_name_past,operating_system_past,os_version_past,browser_past,region_past,city_past,ft_campaign_past,ft_medium_past,ft_source_past,utm_term_placement_past,ProspectActivityName_Max,mx_Last_Activity_done_by_Agent,mx_Sub_Dispostion
0,0.000000,3,3,0,3,2,4,68,0,0,0,0,9,3,46
1,1.474086,2,0,0,0,2,30,47,15,1,6,0,10,5,40
2,0.188153,2,0,0,0,2,30,47,15,1,6,0,10,5,40
3,1.897437,2,0,0,0,2,30,47,15,1,6,0,10,5,40
4,21.324201,2,0,0,0,2,30,47,15,1,6,0,10,5,40


In [4]:
# %%
import pickle
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import chi2
from sklearn import preprocessing
from google.cloud import bigquery
from sklearn.utils import resample
from sklearn.utils import class_weight
from datetime import datetime, timedelta
from scipy.stats import chi2_contingency
from catboost import CatBoostClassifier, Pool
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics  import accuracy_score, recall_score,confusion_matrix, precision_score,roc_auc_score, roc_curve
from sklearn import metrics
import pytz
import os
import glob
from google.cloud import storage
import warnings
import pandas as pd
import numpy as np
import os
import glob
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.model_selection import train_test_split,GridSearchCV
import pickle
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.metrics import accuracy_score, roc_curve, precision_score, recall_score, confusion_matrix, roc_auc_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import json
# from sklearn.ensemble import CatBoostClassifier
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.font_manager
from pyod.models.knn import KNN 
from pyod.utils.data import generate_data, get_outliers_inliers
warnings.simplefilter("ignore")

# %%

print("------------------------------------------------------------------")
fmt = "%Y-%m-%d %H:%M:%S"
now_utc = datetime.now(pytz.timezone('UTC'))
print("------------------------------------------------")
print("UTC time------", now_utc.strftime(fmt))
print(
    "start --------------------",
    now_utc.astimezone(
        pytz.timezone('Asia/Kolkata')).strftime(fmt))

seed = 100

client = bigquery.Client()

print("Client creating using default project: {}".format(client.project))
query = """
  
  with curr_data as (
  SELECT * FROM `jm-ebg.JM_PredictN_Production.jm_predictN_cur_session_his_agg_data_esd`
),
cpd_data as (

  SELECT GAID_Cr, ProspectID_LeadCreation, Source_LeadUpdate as source, 
  SourceMedium_LeadUpdate as medium, SourceCampaign_LeadUpdate as campaign, 
  mx_AOF_Stage_LeadUpdate as mx_AOF_Stage, mx_DIY_Stage_LeadUpdate as mx_DIY_Stage,EmailAddress_LeadUpdate as email,
  Phone_LeadUpdate as Phone,ProspectActivityName_Max_LeadUpdate as ProspectActivityName_Max, 
  mx_Last_Activity_done_by_Agent_LeadUpdate as mx_Last_Activity_done_by_Agent, 
  mx_Sub_Dispostion_LeadUpdate  as mx_Sub_Dispostion from `jm-ebg.cdp_staging.LSQ-LeadMaster`
)

select *except(GAID_Cr, ProspectID_LeadCreation) from curr_data
left join (select * from cpd_data)
on ProspectId = ProspectID_LeadCreation
"""
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    # location="US",
)  # API request - starts the query
df = query_job.to_dataframe()

print("\nchanged df")
print(df.loc[df["ad_group"]=="%7Badgroupid%7D"])
data=pickle.load(open('/home/jupyter/jm_predictN/Tatvic_approach/production/model/cat_dict_esd_RF.pkl', 'rb'))
model = pickle.load(open('/home/jupyter/jm_predictN/Dhawal/model/Dhawal_v1.pkl', 'rb'))
### list of column name dtype wise
string_column = list(df.select_dtypes(include=['O']).columns)
int_column = list(df.select_dtypes(include=['int64']).columns)
float_column = list(df.select_dtypes(include=['float64']).columns)
cat_cols = list(df.select_dtypes(include=['O']).columns)

# df['placement'].loc[df['placement'].apply(lambda x: (x not in data['placement']) )] = '(Others)'
df['device_category_past'].loc[df['device_category_past'].apply(lambda x: (x not in data['device_category_past']) )] = '(Others)'
df['mobile_brand_name_past'].loc[df['mobile_brand_name_past'].apply(lambda x: (x not in data['mobile_brand_name_past']) )] = '(Others)'
df['operating_system_past'].loc[df['operating_system_past'].apply(lambda x: (x not in data['operating_system_past']) )] = '(Others)'
df['os_version_past'].loc[df['os_version_past'].apply(lambda x: (x not in data['os_version_past']) )] = '(Others)'
df['browser_past'].loc[df['browser_past'].apply(lambda x: (x not in data['browser_past']) )] = '(Others)'
df['city_past'].loc[df['city_past'].apply(lambda x: (x not in data['city_past']) )] = '(Others)'
df['region_past'].loc[df['region_past'].apply(lambda x: (x not in data['region_past']) )] = '(Others)'
df['ft_campaign_past'].loc[df['ft_campaign_past'].apply(lambda x: (x not in data['ft_campaign_past']) )] = '(Others)'
df['ft_medium_past'].loc[df['ft_medium_past'].apply(lambda x: (x not in data['ft_medium_past']) )] = '(Others)'
df['ft_source_past'].loc[df['ft_source_past'].apply(lambda x: (x not in data['ft_source_past']) )] = '(Others)'
df['utm_term_placement_past'].loc[df['utm_term_placement_past'].apply(lambda x: (x not in data['utm_term_placement_past']) )] = '(Others)'
# df['placement_past'].loc[df['placement_past'].apply(lambda x: (x not in data['placement_past']) )] = '(Others)''

### replace the string int and float value
for i in string_column:
    df[i].fillna("Value_Not_Found", inplace = True)
for i in int_column:
    df[i].fillna(-99999, inplace = True) 
for i in float_column:
    df[i].fillna(-99999999.000, inplace = True) 

# %%
df = df.drop_duplicates(keep='first').reset_index(drop=True)
predict_info=df[['ProspectId', 'user_pseudo_id', 'visitStartTime', 'event_date','email',
                 'Phone','source', 'medium', 'campaign']]
# %%
drop_cols = ['ProspectId', 'user_pseudo_id', 'visitStartTime', 'event_date', 'day_of_week', 'timeOnSite', 
             'device_category', 'mobile_brand_name', 'operating_system', 'os_version', 'browser', 'region', 
             'city', 'ft_campaign', 'ft_medium', 'ft_source', 'ad_network', 'ad_group', 'utm_term_placement', 
             'ProspectID_Past', 'user_pseudo_id_past', 'visitStartTime_past', 'date_past', 
            'device_category_past','ad_network_past', 'ad_group_past', 'source', 'medium', 
             'campaign', 'mx_AOF_Stage', 'mx_DIY_Stage', 'email', 'Phone']
df = df.drop(drop_cols, axis=1)
print(df)
cat_cols = list(df.select_dtypes(include=['O']).columns)
print(cat_cols)
# c_l=['ad_group','ad_group_past']
# cat_cols = cat_cols.drop(c_l, axis=1)
# %%

# cat_cols = cat_cols.remove("ad_group")
# cat_cols_= cat_cols.remove("ad_group_past")
print(cat_cols)
 # le = pickle.load(open("/home/jupyter/jm_predictN/Tatvic_approach/production/label_encode/{}-esd-RF.pkl".format(i), 'rb'))

# print(cat_cols)
#categorical value handling
le = LabelEncoder()
for  i in cat_cols:
    df[i] = le.fit_transform(df[i])
    pickle.dump(le, open("/home/jupyter/jm_predictN/Dhawal/label_encode/{}-esd-RF.pkl".format(i), 'wb'))
## datatype as training
# feature_dtype = pd.read_csv('/home/jupyter/jm_predictN/Tatvic_approach/production/model/col_dtypes_esd_RF.csv')
# feature_type = {}
# for idx, rows in feature_dtype.iterrows():
#     df[rows['column_name']] = df[rows['column_name']].astype(rows['dtypes'])
#     feature_type[rows['column_name']] = rows['dtypes']

# cat_columns = list(df.dtypes[df.dtypes == 'O'].index)
# print (cat_columns)
# all_columns = list(df.columns)
# cat_idxs = []
# for column in cat_columns:
#     cat_idxs.append(all_columns.index(column))
# print(df.info())
# print(feature_type)
# df = df.astype(feature_type)
# df = df[['timeOnSite', 'device_category', 'mobile_brand_name',
#        'operating_system', 'os_version', 'region', 'city', 'ft_medium',
#        'ft_source', 'sw_medium', 'sw_source', 'mobile_entered',
#        'mobile_verified', 'timeOnSite_past', 'device_category_past',
#        'mobile_brand_name_past', 'operating_system_past', 'os_version_past',
#        'region_past', 'city_past', 'ft_medium_past', 'ft_source_past',
#        'sw_medium_past', 'sw_source_past', 'mobile_entered_past',
#        'mobile_verified_past']]
# drop_cols = ['ad_network','ad_network_past']
#'ProspectId' 'user_pseudo_id' 'visitStartTime' 'event_date'\n 'ProspectID_Past' 'user_pseudo_id_past' 'visitStartTime_past' 'date_past'\n 'user_psudo_id_conv' 'ProspectId_conv' 'date_conv' 'ad_group'\n 'ad_group_past'
# df['ad_group']=df.ad_group.replace('',np.nan).astype(float)

# df["device_category_past"]=df['device_category_past'].astype(float)# df = df.drop(drop_cols, axis=1)

preds_train = model.predict_proba(df)
predict_info['score'] = preds_train[:, 1]
prediction_datetime = (now_utc.astimezone(pytz.timezone('Asia/Kolkata'))).strftime(fmt)
print("date of prediction --", prediction_datetime)

#Decile threshold
lower_threshold=None
upper_threshold=None
# %%
if lower_threshold==None and upper_threshold==None:
    lower_thresh = predict_info['score'].quantile(0.3)
    higher_thresh = predict_info['score'].quantile(0.7)
else:
    lower_thresh = float(lower_threshold)
    higher_thresh = float(upper_threshold)

# %%
print ("Lower Threshold = ", lower_thresh)
print ("Upper Threshold = ", higher_thresh)


# %%


# create bucket
predict_info['bucket'] = np.where(predict_info['score'] >= higher_thresh, 1, 0)
predict_info['bucket'] = np.where(((predict_info['score'] >= lower_thresh) & (
            predict_info['score'] < higher_thresh)),2,predict_info['bucket'])
predict_info['bucket'] = np.where(predict_info['score'] <= lower_thresh,3,
    predict_info['bucket'])
# predict_info['prediction_datetime'] = prediction_datetime
predict_info['product_category'] = 'JM_PredictN'

try : 
    pred_date_bg_tablename = (
            now_utc.astimezone(
                pytz.timezone('Asia/Kolkata'))).strftime("%Y%m%d")
    query = """
        SELECT * FROM `jm-ebg.JM_PredictN_Production.jm_predictN_prediction_esd_{}`  
        """.format(pred_date_bg_tablename)


    query_job = client.query(
            query
        )  

    bq_df_exclude = query_job.to_dataframe()
except : 
    bq_df_exclude = pd.DataFrame(columns =["ProspectId", "user_pseudo_id", "visitStartTime", "event_date","email","Phone", "score", "bucket", "product_category",'source', 'medium', 'campaign', 'mx_AOF_Stage', 'mx_DIY_Stage'])

print("bq_df_exclude shape",bq_df_exclude.shape)
prediction_df_bg = predict_info.sort_values('score', ascending=False)
# prediction_df_bg = prediction_df_bg.drop_duplicates(
#     subset=['ProspectId','event_date'],
#     keep='first').reset_index(
#     drop=True)

print("unique data of prospectid ---", prediction_df_bg.shape)
if not bq_df_exclude.empty:
    prediction_df_bg = pd.concat([prediction_df_bg, bq_df_exclude]).reset_index(drop=True)
    prediction_df_bg = prediction_df_bg.drop_duplicates(subset=['ProspectId','event_date'],keep=False).reset_index(drop=True)
    ### only consider real time data
    # prediction_df_bg = prediction_df_bg.loc[prediction_df_bg['prediction_datetime'] == prediction_datetime].reset_index(drop=True)
else:
    print('Empty BQ dataframe')
    pass


print("exclude previous session---",prediction_df_bg.shape)
prediction_df_bg['event_date']  = prediction_df_bg['event_date'].astype('str')
print("dataframe shape bigquery", prediction_df_bg.shape)
print("Bigquery intertion start")
if not prediction_df_bg.empty:
    # timedelta(days=5) timedelta(minutes=30)
    pred_date_bg_tablename = str(predict_info.event_date.unique()[0])
    # frist method - days wise table create
    # print("prediction_df_bg", prediction_df_bg.shape())
    bq_client = bigquery.Client()
    dataset_ref = bq_client.dataset('Hyperparameter_Tuned_Prediction')
    job_config = bigquery.LoadJobConfig(schema=[
        bigquery.SchemaField(name="ProspectId", field_type="STRING"),
        bigquery.SchemaField(name="user_pseudo_id", field_type="STRING"),
        bigquery.SchemaField(name="visitStartTime", field_type="INTEGER"),
        bigquery.SchemaField(name="event_date", field_type="STRING"),
        bigquery.SchemaField(name="email", field_type="STRING"),
        bigquery.SchemaField(name="Phone", field_type="STRING"),
        bigquery.SchemaField(name="source", field_type="STRING"),
        bigquery.SchemaField(name="medium", field_type="STRING"),
        bigquery.SchemaField(name="campaign", field_type="STRING"),
        bigquery.SchemaField(name="score", field_type="FLOAT"),
        bigquery.SchemaField(name="bucket", field_type="INTEGER"),
        # bigquery.SchemaField(name="prediction_datetime",field_type="STRING"),
        bigquery.SchemaField(name="product_category", field_type="STRING")
    ])
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND
    job_config.create_disposition = bigquery.CreateDisposition.CREATE_IF_NEEDED

    table_ref = dataset_ref.table("JM_Improved_Recall"+pred_date_bg_tablename)
    dataframe = pd.DataFrame(prediction_df_bg.to_records(index=False))
    job = bq_client.load_table_from_dataframe(
        dataframe, table_ref, job_config=job_config)
    job.result()  # Waits for table load to complete.
else:
    pass
print("Bigquery intertion end")


------------------------------------------------------------------
------------------------------------------------
UTC time------ 2023-05-22 05:42:53
start -------------------- 2023-05-22 11:12:53
Client creating using default project: jm-ebg

changed df
Empty DataFrame
Columns: [ProspectId, user_pseudo_id, visitStartTime, event_date, day_of_week, timeOnSite, device_category, mobile_brand_name, operating_system, os_version, browser, region, city, ft_campaign, ft_medium, ft_source, ad_network, ad_group, utm_term_placement, ProspectID_Past, user_pseudo_id_past, visitStartTime_past, date_past, timeOnSite_past, day_of_week_past, device_category_past, mobile_brand_name_past, operating_system_past, os_version_past, browser_past, region_past, city_past, ft_campaign_past, ft_medium_past, ft_source_past, ad_network_past, ad_group_past, utm_term_placement_past, source, medium, campaign, mx_AOF_Stage, mx_DIY_Stage, email, Phone, ProspectActivityName_Max, mx_Last_Activity_done_by_Agent, mx_Sub_Di

In [8]:
df.head()

,day_of_week,timeOnSite,device_category,mobile_brand_name,operating_system,os_version,browser,region,city,ft_campaign,ft_medium,ft_source,utm_term_placement
0,5,0.253620,0,1,1,7,1,6,17,5,0,0,1
1,5,0.636725,0,1,1,7,1,6,17,5,0,0,1
2,5,0.016306,0,1,1,7,1,6,17,5,0,0,1
3,5,0.082931,0,1,1,7,1,6,17,5,0,0,1
4,5,0.067462,0,1,1,7,1,6,17,5,0,0,1
